### Dataset ImageFloder

In [22]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [23]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

### Model

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Preprocess

In [15]:
import os

os.listdir('../dataset/mnist_png/training/')

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [16]:
train_dir = '../dataset/mnist_png/training/'
test_dir = '../dataset/mnist_png/testing/'

In [17]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [18]:
train_dataset = datasets.ImageFolder(root=train_dir,
                                     transform=transforms.Compose([
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.1307,), (0.3081,))
                                     ]))

test_dataset = datasets.ImageFolder(root=test_dir,
                                    transform=transforms.Compose([
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.1307,), (0.3081,))
                                     ]))

In [19]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

### Optimization

In [20]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(),
                      lr=lr,
                      momentum=momentum)

### Training

In [21]:
for epoch in range(1, epochs + 1):
    # Train mode
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # backpropagation 계산 전 0으로 기울기 초기화
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch : {} [{} / {} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()
            ))
            
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)
    ))

Train Epoch : 1 [0 / 60000 (0%)]	Loss : 2.298499
Train Epoch : 1 [6400 / 60000 (11%)]	Loss : 2.247866
Train Epoch : 1 [12800 / 60000 (21%)]	Loss : 2.118407
Train Epoch : 1 [19200 / 60000 (32%)]	Loss : 1.906714
Train Epoch : 1 [25600 / 60000 (43%)]	Loss : 1.300121
Train Epoch : 1 [32000 / 60000 (53%)]	Loss : 0.830375
Train Epoch : 1 [38400 / 60000 (64%)]	Loss : 0.639640
Train Epoch : 1 [44800 / 60000 (75%)]	Loss : 0.401675
Train Epoch : 1 [51200 / 60000 (85%)]	Loss : 0.562704
Train Epoch : 1 [57600 / 60000 (96%)]	Loss : 0.334808

Test set : Average loss : 0.3848, Accuracy : 8888/10000 (89%)

Train Epoch : 2 [0 / 60000 (0%)]	Loss : 0.608515
Train Epoch : 2 [6400 / 60000 (11%)]	Loss : 0.298352
Train Epoch : 2 [12800 / 60000 (21%)]	Loss : 0.527408
Train Epoch : 2 [19200 / 60000 (32%)]	Loss : 0.296251
Train Epoch : 2 [25600 / 60000 (43%)]	Loss : 0.500392
Train Epoch : 2 [32000 / 60000 (53%)]	Loss : 0.202580
Train Epoch : 2 [38400 / 60000 (64%)]	Loss : 0.307653
Train Epoch : 2 [44800 / 60000